# Datasets Generation

Here we create the needed datasets. For each season we'll create two datasets. One with the matches and betting odds and a simple dataset which will contain the season scoreboard.

In [ ]:
import pandas as pd
from pathlib import Path

DATA = Path("data/raw")
OUTPUT = DATA.parent / "processed"

columns_of_interest = [
    "Date",
    "HomeTeam",
    "AwayTeam",
    "FTR",  # Full Time Result
    "B365H",  # Bet365 home win odds
    "B365D",  # Bet365 draw odds
    "B365A",  # Bet365 away win odds
]
datasets = [
    pd.read_csv(f) for f in filter(lambda x: x.suffix == ".csv", DATA.iterdir())
]


Handling the missing values

In [ ]:
datasets = [d.dropna(axis=0, how="all") for d in datasets]


In [ ]:
def fill_missing_odds(df):
    """Some odds might be missing for B365 so we fill them with the ones from another bookmaker."""

    def safe_fill(df, target, fallback_cols):
        if target not in df:
            df[target] = pd.NA

        for col in fallback_cols:
            if col in df:
                df[target] = df[target].fillna(df[col])
        return df

    missing = df[df[["B365H", "B365D", "B365A"]].isna().any(axis=1)]
    if not missing.empty:
        print(
            f"Rows with missing odds\n: {missing[["Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A"]]}",
            end="\n\n",
        )

        df = safe_fill(
            df,
            "B365H",
            [
                "AvgH",
                "MaxH",
                "BbMxH",
                "WHH",
                "BWH",
                "VCH",
                "IWH",
                "BbAvH",
                "LBH",
                "BFEH",
                "B365AHH",
                "PH",
            ],
        )
        df = safe_fill(
            df,
            "B365D",
            [
                "AvgD",
                "MaxD",
                "BbMxD",
                "WHD",
                "BWD",
                "VCD",
                "IWD",
                "BbAvD",
                "LBD",
                "BFED",
                "B365AHD",
                "PD",
            ],
        )
        df = safe_fill(
            df,
            "B365A",
            [
                "AvgA",
                "MaxA",
                "BbMxA",
                "WHA",
                "BWA",
                "VCA",
                "IWA",
                "BbAvA",
                "LBA",
                "BFEA",
                "B365AHA",
                "PA",
            ],
        )

        missing = df[df[["B365H", "B365D", "B365A"]].isna().any(axis=1)]
        if not missing.empty:
            print(
                f"Rows with still missing odds after filling\n: {missing[["Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A"]]}",
                end="\n\n",
            )

            for idx, row in missing.iterrows():
                home_team = row["HomeTeam"]
                away_team = row["AwayTeam"]

                reverse_fixture = df[
                    (df["HomeTeam"] == away_team) & (df["AwayTeam"] == home_team)
                ]

                if not reverse_fixture.empty:
                    if pd.isna(df.loc[idx, "B365H"]):
                        df.loc[idx, "B365H"] = reverse_fixture["B365A"].values[0]

                    if pd.isna(df.loc[idx, "B365D"]):
                        df.loc[idx, "B365D"] = reverse_fixture["B365D"].values[0]

                    if pd.isna(df.loc[idx, "B365A"]):
                        df.loc[idx, "B365A"] = reverse_fixture["B365H"].values[0]

            missing = df[df[["B365H", "B365D", "B365A"]].isna().any(axis=1)]
            if not missing.empty:
                raise Exception(
                    "There are still missing odds after filling with other bookmakers. Please check the data."
                )
        else:
            print(f"All missing odds filled successfully.", end="\n\n")
    else:
        print(f"No missing odds found.", end="\n\n")
    return df


datasets = [fill_missing_odds(d) for d in datasets]


Rows with missing odds
:           Date HomeTeam AwayTeam  B365H  B365D  B365A
35  18/10/2020  Udinese    Parma    NaN    NaN    NaN
37  19/10/2020   Verona    Genoa    NaN    NaN    NaN

All missing odds filled successfully.

No missing odds found.

No missing odds found.

No missing odds found.

No missing odds found.

No missing odds found.

No missing odds found.

No missing odds found.

Rows with missing odds
:            Date HomeTeam    AwayTeam  B365H  B365D  B365A
203  10/01/2022   Torino  Fiorentina    NaN    NaN    NaN

Rows with still missing odds after filling
:            Date HomeTeam    AwayTeam  B365H  B365D  B365A
203  10/01/2022   Torino  Fiorentina    NaN    NaN    NaN

No missing odds found.

No missing odds found.

Rows with missing odds
:          Date HomeTeam AwayTeam  B365H  B365D  B365A
325  17/04/11   Chievo  Bologna    NaN    NaN    NaN
327  17/04/11    Genoa  Brescia    NaN    NaN    NaN

All missing odds filled successfully.

No missing odds found.

No mi

In [ ]:
def extract_leaderboard(df):
    """
    Extract the leaderboard from the dataframe.

    Note: Takes into account only the Full Time Result (FTR) column.
    Real leaderboards are more complex and take into account other factors like goal difference, goals scored, etc.
    This is a simplified version is enough.
    """
    teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))
    leaderboard = pd.DataFrame(teams, columns=["Team"])
    leaderboard["Points"] = 0
    for _, row in df.iterrows():
        if row["FTR"] == "H":
            leaderboard.loc[leaderboard["Team"] == row["HomeTeam"], "Points"] += 3
        elif row["FTR"] == "A":
            leaderboard.loc[leaderboard["Team"] == row["AwayTeam"], "Points"] += 3
        elif row["FTR"] == "D":
            leaderboard.loc[leaderboard["Team"] == row["HomeTeam"], "Points"] += 1
            leaderboard.loc[leaderboard["Team"] == row["AwayTeam"], "Points"] += 1
    leaderboard = leaderboard.sort_values(by="Points", ascending=False)
    leaderboard.reset_index(drop=True, inplace=True)
    leaderboard.index += 1
    leaderboard.index.name = "Position"
    leaderboard.index = leaderboard.index.astype(int)
    return leaderboard


Store the datasets

In [ ]:
datasets = [d[columns_of_interest] for d in datasets]
datasets = [
    d.rename(columns={"B365H": "HomeOdds", "B365D": "DrawOdds", "B365A": "AwayOdds"})
    for d in datasets
]

for df in datasets:
    filename = df.iloc[0]["Date"].split("/")[-1]
    filename = "20" + filename if len(filename) == 2 else filename
    filename = f"{filename}-{int(filename[-2:])+1:02}"
    df.to_csv(OUTPUT / f"season_{filename}.csv", index=False)
    leaderboard = extract_leaderboard(df)
    leaderboard.to_csv(OUTPUT / f"leaderboard_{filename}.csv", index=False)
